In [8]:
import numpy as np
import pandas as pd
import sys
import requests

import json

BATTLEID_TITLE = 'BattleId'
JOBID_TITLE = 'JobId'
SKILL_TITLE = 'Skills'
CHARACTER_TITLE = 'Characters'

RETURN_TIME = 'Milliseconds'
RETURN_RESULT = 'result'
RETURN_DAMAGE = 'Damage'

global battle_id
battle_id = 0
population_warning = True

population_size = 100
job_to_skills = {}


In [9]:
def read_data(file_path):
    global battle_id
    global job_to_skills
    print(f"reading data from {file_path}...")
    xls = pd.ExcelFile(file_path)
    jobs = pd.read_excel(xls, 'job')
    skills = pd.read_excel(xls, 'skill')

    # just take the first 7
    for i in range(8):
        job_id = jobs.loc[:7, 'job_id'][i]
        skills_id = []
        job_to_skills[job_id] = skills_id

    # for skill in skills.loc:
    for index, row in skills.iterrows():
        skill_id = row['id']
        job_id = row['job_id']
        assert job_id in job_to_skills, f"job_id {job_id} not in job_to_skills"
        job_to_skills[job_id].append(skill_id)

"""
generates a random population of size n
sample template: '[{"BattleId":1,"Characters":[{"JobId":14,"Skills":[109,110,111,112]},{"JobId":15,"Skills":[104,103,106,107]},{"JobId":16,"Skills":[446,448,449,447]},{"JobId":17,"Skills":[1116,1119,1117,1118]}]}]'
"""
def generate_first_population(population_size):
    global battle_id
    global job_to_skills
    population = []
    for i in range(population_size):
        population.append(generate_character())
    return population, generate_id_to_idx(population)

"""
generates a random character
sample template: {"BattleId":1,"Characters":[{"JobId":14,"Skills":[109,110,111,112]},{"JobId":15,"Skills":[104,103,106,107]},{"JobId":16,"Skills":[446,448,449,447]},{"JobId":17,"Skills":[1116,1119,1117,1118]}]}
"""
def generate_character():
    global battle_id
    global job_to_skills
    character = {}
    character[BATTLEID_TITLE] = battle_id
    battle_id += 1
    character[CHARACTER_TITLE] = []
    job_ids = np.random.choice(list(job_to_skills.keys()), 4, replace=False)
    for job_id in job_ids:
        job = {}
        job[JOBID_TITLE] = job_id
        job[SKILL_TITLE] = np.random.choice(job_to_skills[job_id], 4, replace=False).tolist()
        character[CHARACTER_TITLE].append(job)
    return character

def generate_id_to_idx(population):
    new_id_to_idx = {}
    for i, battle in enumerate(population):
        new_id_to_idx[battle[BATTLEID_TITLE]] = i
    return new_id_to_idx

In [3]:
read_data('job-skill.xlsx')
population, id_to_idx = generate_first_population(10)

print(population)

reading data from job-skill.xlsx...
[{'BattleId': 0, 'Characters': [{'JobId': 25, 'Skills': [854, 918, 582, 138]}, {'JobId': 14, 'Skills': [1375, 1371, 1948, 1955]}, {'JobId': 15, 'Skills': [1297, 1304, 241, 1248]}, {'JobId': 16, 'Skills': [1241, 1637, 449, 1960]}]}, {'BattleId': 1, 'Characters': [{'JobId': 18, 'Skills': [1032, 1043, 1040, 1706]}, {'JobId': 15, 'Skills': [1938, 1564, 438, 1620]}, {'JobId': 14, 'Skills': [1376, 1364, 1340, 1666]}, {'JobId': 25, 'Skills': [866, 865, 394, 875]}]}, {'BattleId': 2, 'Characters': [{'JobId': 14, 'Skills': [444, 1327, 1358, 1361]}, {'JobId': 19, 'Skills': [134, 957, 131, 942]}, {'JobId': 26, 'Skills': [786, 809, 803, 1702]}, {'JobId': 16, 'Skills': [1641, 1637, 1231, 118]}]}, {'BattleId': 3, 'Characters': [{'JobId': 15, 'Skills': [1306, 1262, 1566, 1299]}, {'JobId': 25, 'Skills': [869, 898, 874, 895]}, {'JobId': 18, 'Skills': [1038, 1010, 1026, 1057]}, {'JobId': 17, 'Skills': [1110, 1139, 1576, 1091]}]}, {'BattleId': 4, 'Characters': [{'JobId'

In [11]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [12]:
url = 'http://10.41.0.159:8080/simulator'

headers = {
    'Content-Type': 'application/json'
}

population_j = json.dumps(population, cls=NpEncoder)
loaded_population = json.loads(population_j)
data = loaded_population

response = requests.post(url, json=data, headers=headers)

print(response.text)

[{"BattleId":7,"Result":0,"Milliseconds":47860,"Damage":3706668322},{"BattleId":5,"Result":0,"Milliseconds":63860,"Damage":5180549905},{"BattleId":2,"Result":0,"Milliseconds":72370,"Damage":5755266194},{"BattleId":4,"Result":0,"Milliseconds":74650,"Damage":6396612675},{"BattleId":6,"Result":0,"Milliseconds":70200,"Damage":7337596248},{"BattleId":9,"Result":0,"Milliseconds":44520,"Damage":3511765937},{"BattleId":3,"Result":0,"Milliseconds":75770,"Damage":5527369055},{"BattleId":0,"Result":0,"Milliseconds":85940,"Damage":5792862864},{"BattleId":1,"Result":0,"Milliseconds":73930,"Damage":4147769428},{"BattleId":8,"Result":0,"Milliseconds":74090,"Damage":6175754373}]


In [17]:
len(response.json())

10

In [18]:
response = [{"BattleId":7,"Result":0,"Milliseconds":47860,"Damage":3706668322},{"BattleId":5,"Result":0,"Milliseconds":63860,"Damage":5180549905},{"BattleId":2,"Result":0,"Milliseconds":72370,"Damage":5755266194},{"BattleId":4,"Result":0,"Milliseconds":74650,"Damage":6396612675},{"BattleId":6,"Result":0,"Milliseconds":70200,"Damage":7337596248},{"BattleId":9,"Result":0,"Milliseconds":44520,"Damage":3511765937},{"BattleId":3,"Result":0,"Milliseconds":75770,"Damage":5527369055},{"BattleId":0,"Result":0,"Milliseconds":85940,"Damage":5792862864},{"BattleId":1,"Result":0,"Milliseconds":73930,"Damage":4147769428},{"BattleId":8,"Result":0,"Milliseconds":74090,"Damage":6175754373}]

all_damages = [i['Damage'] for i in response]
print(max(all_damages))

7337596248


In [ ]:
16692099104 - 7344716581

-111366815